# 잠재요인 협업 필터링 SVD

 - Surprise를 이용한 영화 추천 알고리즘

In [1]:
import pandas as pd
import numpy as np
from google.colab import files
up = files.upload()

Saving movies.csv to movies.csv
Saving ratings.csv to ratings.csv
Saving ratings_noh.csv to ratings_noh.csv
Saving README.txt to README.txt
Saving tags.csv to tags.csv


In [2]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
ratings.shape

(100836, 4)

In [4]:
# 사용자수, 영화 개수
ratings.userId.nunique(), ratings.movieId.nunique()

(610, 9724)

In [5]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 5.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630142 sha256=2aa388c7d1b9a369ed48e7a014db0e37eed51d05e845abf08062e47f6e9b94ef
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [6]:
from surprise import SVD, Reader
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep = ',', rating_scale = (0.5, 5))
data_folds = DatasetAutoFolds('ratings_noh.csv', reader = reader)

In [7]:
# 전체 데이터를 학습 데이터로 활용
trainset =data_folds.build_full_trainset()

In [8]:
# 모델 생성 및 학습
model = SVD(n_epochs=20, n_factors=50, random_state=2022)
model.fit(trainset)

In [9]:
# 영화 정보
mdf = pd.read_csv('movies.csv')
mdf.head(40).tail(5)

,movieId,title,genres
35,39,Clueless (1995),Comedy|Romance
36,40,"Cry, the Beloved Country (1995)",Drama
37,41,Richard III (1995),Drama|War
38,42,Dead Presidents (1995),Action|Crime|Drama
39,43,Restoration (1995),Drama


In [10]:
# 9번 유저가 42번 영화를 봤는지 확인
movieIds = ratings[ratings.userId == 9]['movieId']
movieIds[movieIds == 42].count()

0

In [11]:
# 9번 유저가 42번 영화의 예상 평점
uid, mid = str(9), str(42)
pred=model.predict(uid, mid, verbose= True)

user: 9          item: 42         r_ui = None   est = 3.25   {'was_impossible': False}


In [12]:
pred

Prediction(uid='9', iid='42', r_ui=None, est=3.249924377339538, details={'was_impossible': False})

 - 사용자 9번이 보지 않은 영화 중에서 예상점수가 높은 top 10

In [13]:
seen_mov = ratings[ratings.userId == 9]['movieId'].tolist()
total_mov = mdf.movieId.tolist()
un_mov = [movie for movie in total_mov if movie not in seen_mov]
len(seen_mov), len(un_mov)

(46, 9696)

In [14]:
uid = str(9)
# 전체 영화 리스트 중에 9번 영화가 안 본 영화를 조회
# predictions = [model.predict(uid, str(mid)) for mid in un_mov]
predictions = []

for mid in un_mov:
    pred = model.predict(uid, str(mid))
    predictions.append(pred)

predictions[:5]

[Prediction(uid='9', iid='1', r_ui=None, est=3.702922347424712, details={'was_impossible': False}),
 Prediction(uid='9', iid='2', r_ui=None, est=3.2274451421980412, details={'was_impossible': False}),
 Prediction(uid='9', iid='3', r_ui=None, est=3.0342513115122123, details={'was_impossible': False}),
 Prediction(uid='9', iid='4', r_ui=None, est=2.661778597408914, details={'was_impossible': False}),
 Prediction(uid='9', iid='5', r_ui=None, est=2.689490348191407, details={'was_impossible': False})]

In [15]:
# prediction 내 est 값 순서로 정렬시키는 함수
def sort_key_est(pred):
    return pred.est

In [16]:
predictions.sort(key = sort_key_est, reverse = True)
predictions[:5]

[Prediction(uid='9', iid='318', r_ui=None, est=4.070330794979969, details={'was_impossible': False}),
 Prediction(uid='9', iid='1217', r_ui=None, est=4.063731956995097, details={'was_impossible': False}),
 Prediction(uid='9', iid='1261', r_ui=None, est=4.051908410348554, details={'was_impossible': False}),
 Prediction(uid='9', iid='1204', r_ui=None, est=4.0227662213503805, details={'was_impossible': False}),
 Prediction(uid='9', iid='3275', r_ui=None, est=4.011500870494226, details={'was_impossible': False})]

In [17]:
top_mid = [int(pred.iid) for pred in predictions[:10]] # top 10 영화 목록
top_mr = [int(pred.est) for pred in predictions[:10]] # top 10 평점 순
top_mt = [mdf[mdf.movieId == mid]['title'] for mid in top_mid]

In [18]:
top_df = pd.DataFrame({
    '영화명': top_mt,
    '예상평점': top_mr
})
top_df.index.name = 'mdf_index'
top_df

,영화명,예상평점
mdf_index,,
0,"277 Shawshank Redemption, The (1994) Name: ...",4
1,"918 Ran (1985) Name: title, dtype: object",4
2,960 Evil Dead II (Dead by Dawn) (1987) Name...,4
3,"906 Lawrence of Arabia (1962) Name: title, ...",4
4,"2462 Boondock Saints, The (2000) Name: titl...",4
5,3622 Amelie (Fabuleux destin d'Amélie Poula...,3
6,"9071 Spotlight (2015) Name: title, dtype: o...",3
7,"1258 Boogie Nights (1997) Name: title, dtyp...",3
8,"46 Usual Suspects, The (1995) Name: title, ...",3
